In [1]:
import numpy as np
import pandas as pd
import spacy
from multiprocessing import Pool
import re

In [2]:
train = pd.read_pickle("../input_data/input.pkl")

In [3]:
nlp = spacy.load('es')

In [4]:
train.head()

,question,answer,story,story_length,question_length
6549,VIAJE AL REINO HERMÉTICO: Una española en @pla...,@entity80,"En @entity388 se aprende a llorar en silencio,...",2000,9
6550,El estudiante de EEUU detenido en @placeholder...,@entity212,"En @entity212 se aprende a llorar en silencio,...",2000,13
8163,Volver a nacer dos veces en un mismo siglo en ...,@entity2,Si se callejea por el casco antiguo de @entity...,1998,11
25466,"\nMientras el @entity458, @placeholder y @enti...",@entity170,"Desde que la crisis se hizo evidente, la @enti...",1993,21
25465,"\nMientras el @placeholder, @entity10 y @entit...",@entity126,"Desde que la crisis se hizo evidente, la @enti...",1993,21


Error 1 (temporarily fixed)

In [26]:
201 * 16

3216

In [28]:
finds = train['story'].apply(lambda x: re.search(r'@entity3995', x))

In [29]:
finds[~pd.isnull(finds)].apply(lambda x: x.group(0))

28017    @entity3995
Name: story, dtype: object

In [30]:
train['story'][28017]

'@entity497 ha lanzado una nueva versión del @entity71 apenas medio año después del estreno del @entity71 6S. En esta ocasión, la compañía se ha decantado por un terminal de cuatro pulgadas que rompe con la tendencia actual del mercado, empeñado en crear terminales de pantalla grande. El nuevo @entity71 @entity446, que acaba de aterrizar en @entity45@entity39, pretende seducir a aquellos usuarios que prefieren un smartphone que pueda manejarse con una sola mano. Pero las dimensiones no son la única arma de seducción del nuevo gadget de los de @entity@entity3977, su precio (desde 489 euros en @entity497) también juega a su favor. \nNo obstante, aunque el precio del teléfono esté por debajo de lo que nos tiene acostumbrados @entity497, lo cierto es que se trata de un desembolso de varios centenares de euros, una cantidad que no todo el mundo puede abonar de golpe. ¿La solución? O bien ahorrar o bien financiar el teléfono. En este último caso, la pregunta es evidente: ¿dónde sale más bara

Error 2:

In [15]:
train.iloc[29521]['question']

'\nLas autoridades de @placeholder han ahogado la protesta estudiantil tras dos meses y medio '

In [10]:
d = nlp(train.iloc[29521]['question'])

In [11]:
d


Las autoridades de @placeholder han ahogado la protesta estudiantil tras dos meses y medio 

In [12]:
[w.has_vector for w in d]

[False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [7]:
# def lengths(x):
#     return len(nlp(x, entity=False, parse=False, tag=False))

# with Pool(4) as p:
#     train['s_len'] = list(p.map(lengths, train['story']))

In [12]:
bn = 461
focus = train.iloc[(bn - 1) * 8: (bn + 20 - 1) * 8, :]

In [17]:
def get_word_ids(doc, rnn_encode=False, max_length=100, nr_unk=100):
    queue = list(doc)
    X = np.zeros(max_length, dtype='int32')
    M = np.zeros(max_length, dtype='int32')
    V = np.zeros(max_length, dtype='int32')
    words = []
    while len(words) <= max_length and queue:
        word = queue.pop(0)
        if rnn_encode or (not word.is_punct and not word.is_space):
            words.append(word)
    words.sort()
    for j, token in enumerate(words):
        if token.text == '@placeholder':
            V[j] = 1
        elif token.text[:7] == '@entity':
            V[j] = int(re.search(r'\d+', token.text[7:]).group(0)) + 2
        if token.has_vector:
            X[j] = token.rank+1
            M[j] = 1
        else:
            X[j] = (token.shape % (nr_unk - 1)) + 2
        j += 1
        if j >= max_length:
            break
    return X, M, V

In [31]:
for i in range(focus.shape[0]):
    X, M, V = get_word_ids(nlp(focus['story'].iloc[i], parse=False, tag=False, entity=False), max_length=2000)
    print(i)
    print(X.max(), X.min())
    print(V.max(), V.min())

0
692207 0
496 0
1
341773 0
475 0
2
692207 0
498 0
3
692207 0
501 0
4
341773 0
487 0
5
525760 0
500 0
6
831419 0
490 0
7
956716 0
466 0
8
850601 0
480 0
9
850601 0
476 0
10
692207 0
468 0
11
453924 0
499 0
12
1158527 0
460 0
13
786991 0
490 0
14
453924 0
501 0
15
453924 0
495 0
16
453924 0
497 0
17
523582 0
487 0
18
453924 0
491 0
19
443588 0
492 0
20
218098 0
496 0
21
218098 0
494 0
22
956716 0
490 0
23
956716 0
501 0
24
218098 0
500 0
25
453924 0
494 0
26
218098 0
501 0
27
688727 0
499 0
28
443588 0
453 0
29
443588 0
445 0
30
971417 0
482 0
31
971417 0
489 0
32
971417 0
499 0
33
218098 0
490 0
34
971417 0
492 0
35
971417 0
496 0
36
971417 0
497 0
37
971417 0
500 0
38
682479 0
494 0
39
956716 0
494 0
40
218098 0
486 0
41
956716 0
500 0
42
383190 0
490 0
43
635482 0
493 0
44
688727 0
491 0
45
726813 0
494 0
46
805189 0
497 0
47
383190 0
497 0
48
805189 0
501 0
49
577985 0
477 0
50
577985 0
495 0
51
541318 0
496 0
52
805189 0
494 0
53
541318 0
495 0
54
523582 0
479 0
55
682479 0
459 0
5